In [ ]:
from pathlib import Path
from PIL import Image
import torch
from torchvision import transforms
from torch import nn
from torch.utils.data import DataLoader,Dataset
import pathlib
from diffusers import DDPMScheduler
from diffusers.optimization import get_cosine_schedule_with_warmup
from diffusers import DDPMPipeline
from diffusers.utils import make_image_grid
import os

from accelerate import Accelerator
from huggingface_hub import create_repo, upload_folder
from tqdm.auto import tqdm
import neptune as neptune

image_path = Path('/kaggle/input/quijote-images')
image_path

In [ ]:
'''from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")
secret_value_0'''

In [2]:
import os

NEPTUNE_API_KEY = os.environ.get('NEPTUNE_API_TOKEN')


'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiNGQzNTc4ZS02YjMwLTRhODktOTE3ZC03NzQ3ZThlNzE5ZWMifQ=='

In [ ]:
    
run = neptune.init_run(
    project="Education/cosmos",
    api_token=NEPTUNE_API_KEY
)


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!sudo apt -qq install git-lfs
!git config --global credential.helper store

In [3]:
from config import TrainingConfig
from dataclasses import asdict


config = TrainingConfig()

run["parameters"] = asdict(config)

In [ ]:
image_transform = transforms.Compose([
        transforms.Resize((config.image_size,config.image_size)),
        transforms.ToTensor(),
        #transforms.Normalize([0.5],[0.5])

])

In [6]:
from customDataClass import CosmosImageData
dataset = CosmosImageData(image_path,transform = image_transform)
train_dataloader  = DataLoader(dataset,batch_size = config.train_batch_size,shuffle=True)


In [12]:
from model import unet_model

model = unet_model(config)

In [ ]:
sample_image = dataset[0].unsqueeze(0)
print("Input shape:", sample_image.shape)

print("Output shape:", model(sample_image, timestep=0).sample.shape)

In [ ]:


noise_scheduler = DDPMScheduler(num_train_timesteps=1000,beta_schedule ='squaredcos_cap_v2')
noise = torch.randn(sample_image.shape)
timesteps = torch.LongTensor([250])
noisy_image = noise_scheduler.add_noise(sample_image, noise, timesteps)

Image.fromarray(((noisy_image.permute(0, 2, 3, 1) + 1.0) * 127.5).type(torch.uint8).numpy()[0])


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * config.num_epochs),
)

In [1]:
from evaluate import sample_images


c:\Users\balak\miniconda3\envs\pytorch2\lib\site-packages\diffusers\models\transformers\transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


In [4]:
from train import train_loop

In [ ]:
from accelerate import notebook_launcher

args = (config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler,run)

notebook_launcher(train_loop, args, num_processes=1)

In [ ]:
torch.cuda.empty_cache()
import gc
torch.cuda.empty_cache()
gc.collect()